<a href="https://colab.research.google.com/github/ekumar0987/Machine-Learning/blob/main/Stock_Closing_Price_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Business Problem

Develop a predictive model to forecast the closing price of a stock on any given day, facilitating data-driven financial analysis for portfolio optimization, risk assessment, and strategy development in algorithmic trading systems. The goal is to leverage historical price and volume data to generate accurate, timely predictions that can support investment decisions and enhance market responsiveness.

### Data Extraction

##### Unzip and load the data

In [1]:
from google.colab import auth
auth.authenticate_user()

In [2]:
import traceback

try:
    from google.colab import drive
    drive.mount('/content/drive', force_remount=True)
except Exception as e:
    traceback.print_exc()

Mounted at /content/drive


In [ ]:
import zipfile
import os
import pandas as pd

print("Current Working Directory: ", os.getcwd())
os.chdir('/content/drive/My Drive/Pinky Personal/Data Science/Data')
print("New Working Directory: ", os.getcwd())

zip_file_path = 'archive.zip'
extract_to_dir = '.'

# Extract the zip file
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extract_to_dir)

Current Working Directory:  /content
New Working Directory:  /content/drive/My Drive/Pinky Personal/Data Science/Data


In [ ]:
import os
print(os.listdir('/content/drive/My Drive/Pinky Personal/Data Science/Data'))

/content
['archive.zip', 'ETFs', 'Stocks']


##### Create a unified financial dataset
Read data from each stock/ETF file and merge it into a single dataset. Augment data with ticker and type information

In [ ]:
import os
import pandas as pd

print(os.getcwd())
os.chdir('/content/drive/My Drive/Pinky Personal/Data Science/Data')

def load_data_from_directory(directory, data_type):
    all_data = []
    for filename in os.listdir(directory):
      if filename.endswith('.txt'):
          file_path = os.path.join(directory, filename)
          if os.stat(file_path).st_size == 0:  # Check if the file is empty
              print(f"Skipping empty file: {filename}")
              continue
          stock_data = pd.read_csv(file_path)
          ticker = filename.split('/')[-1].replace('.us.txt', '')
          stock_data['Ticker'] = ticker
          stock_data['Type'] = data_type  # Add type column (Stock or ETF)
          all_data.append(stock_data)

    return pd.concat(all_data, ignore_index=True)

stocks_data = load_data_from_directory('/content/drive/My Drive/Pinky Personal/Data Science/Data/Stocks', 'Stock')
etfs_data = load_data_from_directory('/content/drive/My Drive/Pinky Personal/Data Science/Data/ETFs', 'ETF')

# Combine the stocks and ETFs data into a single DataFrame
combined_data = pd.concat([stocks_data, etfs_data], ignore_index=True)


/content
Skipping empty file: stnl.us.txt
Skipping empty file: vist.us.txt
Skipping empty file: vmet.us.txt
Skipping empty file: wnfm.us.txt
Skipping empty file: wspt.us.txt
Skipping empty file: pxus.us.txt
Skipping empty file: rbio.us.txt
Skipping empty file: sail.us.txt
Skipping empty file: sbt.us.txt
Skipping empty file: scci.us.txt
Skipping empty file: scph.us.txt
Skipping empty file: send.us.txt
Skipping empty file: sfix.us.txt
Skipping empty file: srva.us.txt
Skipping empty file: molc.us.txt
Skipping empty file: otg.us.txt
Skipping empty file: pbio.us.txt
Skipping empty file: jt.us.txt
Skipping empty file: mapi.us.txt
Skipping empty file: fmax.us.txt
Skipping empty file: gnst.us.txt
Skipping empty file: hayu.us.txt
Skipping empty file: ehr.us.txt
Skipping empty file: amrh.us.txt
Skipping empty file: amrhw.us.txt
Skipping empty file: asns.us.txt
Skipping empty file: bbrx.us.txt
Skipping empty file: bolt.us.txt
Skipping empty file: boxl.us.txt
Skipping empty file: bxg.us.txt
Skippi

##### Exploratory Data Analysis

In [ ]:
import pandas as pd

print(combined_data.head())

print(combined_data.describe())

print(combined_data.info())

         Date  Open  High   Low  Close  Volume  OpenInt Ticker   Type
0  2005-02-25  6.34  6.44  6.34   6.41    3200        0    ssy  Stock
1  2005-02-28  6.43  6.58  6.43   6.58    1500        0    ssy  Stock
2  2005-03-01  6.68  6.98  6.68   6.89    7200        0    ssy  Stock
3  2005-03-02  6.90  6.92  6.90   6.92    1500        0    ssy  Stock
4  2005-03-03  6.90  6.97  6.85   6.91    3000        0    ssy  Stock
               Open          High           Low         Close        Volume  \
count  1.735116e+07  1.735116e+07  1.735116e+07  1.735116e+07  1.735116e+07   
mean   2.640325e+04  2.712965e+04  2.551097e+04  2.628191e+04  1.584260e+06   
std    3.893321e+06  4.005433e+06  3.749236e+06  3.873026e+06  8.424769e+06   
min    0.000000e+00  4.000000e-03 -1.000000e+00  3.700000e-03  0.000000e+00   
25%    8.877900e+00  9.007400e+00  8.731300e+00  8.876200e+00  2.572600e+04   
50%    1.837600e+01  1.862100e+01  1.811300e+01  1.837600e+01  1.573000e+05   
75%    3.366000e+01  3.4060

##### Data Preprocessing

In [ ]:
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split, TimeSeriesSplit
import pandas as pd
import numpy as np
import warnings

warnings.filterwarnings('ignore', message='is_sparse is deprecated and will be removed in a future version.*')

sampled_data = combined_data.sample(frac=0.1, random_state=42)

sampled_data['Datetime'] = pd.to_datetime(sampled_data['Date'])
sampled_data.sort_values(by='Datetime', inplace=True)
sampled_data['Year'] = sampled_data['Datetime'].dt.year
sampled_data['Month'] = sampled_data['Datetime'].dt.month
sampled_data['Day'] = sampled_data['Datetime'].dt.day

# Encode 'Type' using one-hot encoding
sampled_data = pd.get_dummies(sampled_data, columns=['Type'], dtype=int)

# Initialize LabelEncoder
label_encoder = LabelEncoder()
sampled_data['TickerEncoded'] = label_encoder.fit_transform(sampled_data['Ticker'])

# Now drop the original 'Ticker' column
sampled_data.drop('Ticker', axis=1, inplace=True)

# Define y and X from sampled_data
y = sampled_data['Close']
X = sampled_data.drop(['Close', 'Date', 'Datetime', 'OpenInt'], axis=1)

# Define columns to scale (excluding 'Year', 'Month', 'Day', 'TickerEncoded')
columns_to_scale = ['Open', 'High', 'Low', 'Volume']

# Initialize StandardScaler for features and target
scaler_features = StandardScaler()
scaler_target = StandardScaler()

# Prepare lists to store train and test data
train_dfs = []
test_dfs = []

# Define the maximum number of splits for TimeSeriesSplit
max_splits = 5

# Initialize the TimeSeriesSplit with a dynamic split number
tscv = TimeSeriesSplit(n_splits=max_splits)

# Group by 'TickerEncoded' and perform time series split
for ticker, group_data in X.groupby('TickerEncoded'):
    y_group = y[group_data.index].reset_index(drop=True)
    group_data = group_data.reset_index(drop=True)

    # Adjusting n_splits to ensure at least 2 splits if the group size allows
    if len(group_data) > 2:  # Check if group size allows for at least one split
        n_splits = min(max_splits, len(group_data) - 1)
        tscv = TimeSeriesSplit(n_splits=n_splits)

        for train_index, test_index in tscv.split(group_data):
            # Split into train and test sets
            X_train_grp, X_test_grp = group_data.iloc[train_index], group_data.iloc[test_index]
            y_train_grp, y_test_grp = y_group.iloc[train_index], y_group.iloc[test_index]

            # Scale features
            X_train_grp_scaled = scaler_features.fit_transform(X_train_grp[columns_to_scale])
            X_test_grp_scaled = scaler_features.transform(X_test_grp[columns_to_scale])

            # Scale target
            y_train_grp_scaled = scaler_target.fit_transform(y_train_grp.values.reshape(-1, 1)).flatten()
            y_test_grp_scaled = scaler_target.transform(y_test_grp.values.reshape(-1, 1)).flatten()

            X_train_grp = pd.concat([pd.DataFrame(X_train_grp_scaled, columns=columns_to_scale),
                                  X_train_grp.drop(columns=columns_to_scale).reset_index(drop=True)], axis=1)
            X_test_grp = pd.concat([pd.DataFrame(X_test_grp_scaled, columns=columns_to_scale),
                                 X_test_grp.drop(columns=columns_to_scale).reset_index(drop=True)], axis=1)

            # Append to the lists
            train_dfs.append((X_train_grp, y_train_grp_scaled))
            test_dfs.append((X_test_grp, y_test_grp_scaled))

    else:
        # Handle groups with 2 or fewer samples, perhaps by adding them entirely to training or testing
        pass

# Concatenate all training and testing groups
X_train, y_train = zip(*train_dfs)
X_train = pd.concat(X_train, ignore_index=True)
y_train = np.concatenate(y_train, axis=0)

X_test, y_test = zip(*test_dfs)
X_test = pd.concat(X_test, ignore_index=True)
y_test = np.concatenate(y_test, axis=0)

Open             0
High             0
Low              0
Volume           0
Year             0
Month            0
Day              0
Type_ETF         0
Type_Stock       0
TickerEncoded    0
dtype: int64
Open             0
High             0
Low              0
Volume           0
Year             0
Month            0
Day              0
Type_ETF         0
Type_Stock       0
TickerEncoded    0
dtype: int64


In [ ]:
from xgboost import XGBRegressor
from scipy.stats import uniform, randint
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import mean_squared_error, r2_score

# Define a parameter grid to search
param_dist = {
    'n_estimators': randint(50, 1000),
    'max_depth': randint(3, 10),
    'learning_rate': uniform(0.01, 0.2),
    'subsample': uniform(0.6, 0.4),
    'colsample_bytree': uniform(0.6, 0.4),
    'min_child_weight': randint(1, 10),
    'gamma': uniform(0, 0.5),
}

# Initialize the XGBRegressor
xgb = XGBRegressor(random_state=42)

# Initialize the RandomizedSearchCV object
xgb_random_search = RandomizedSearchCV(
    xgb, param_distributions=param_dist,
    n_iter=100,
    scoring='r2',
    cv=3,
    verbose=3,
    random_state=42,
    n_jobs=-1
)

# Fit the RandomizedSearchCV object to the data
xgb_random_search.fit(X_train, y_train)

# Print the best parameters and the corresponding R2
best_score = xgb_random_search.best_score_
best_params = xgb_random_search.best_params_
print(f"Best R2 from Randomized Search: {best_score}")
print(f"Best Parameters: {best_params}")

Fitting 3 folds for each of 100 candidates, totalling 300 fits
Best R2 from Randomized Search: 0.9952126144966934
Best Parameters: {'colsample_bytree': 0.9977829850443283, 'gamma': 0.08796262633867269, 'learning_rate': 0.013615072723104174, 'max_depth': 8, 'min_child_weight': 5, 'n_estimators': 865, 'subsample': 0.8985965620472096}


KeyError: 'mean_test_r2_score'

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import make_scorer, r2_score, mean_squared_error

# Define your parameter grid and model as before

# Create scorers dictionary
scorers = {
    'r2': make_scorer(r2_score),
    'neg_mean_squared_error': make_scorer(mean_squared_error, greater_is_better=False)
}

# Pass the scorers dictionary to the scoring parameter
random_search = RandomizedSearchCV(estimator=xgb,
                                   param_distributions=param_dist,
                                   n_iter=100,
                                   scoring=scorers,
                                   refit='r2',  # Metric to refit the model on
                                   cv=3,
                                   verbose=3,
                                   random_state=42,
                                   n_jobs=-1,
                                   return_train_score=True)

# Fit the RandomizedSearchCV object to the data
random_search.fit(X_train, y_train)

# After fitting, the scores are stored in the cv_results_ attribute
cv_results = random_search.cv_results_

# Access the mean test score for each metric
mean_test_r2 = cv_results['mean_test_r2']
mean_test_neg_mse = cv_results['mean_test_neg_mean_squared_error']

Fitting 3 folds for each of 100 candidates, totalling 300 fits


/usr/local/lib/python3.10/dist-packages/joblib/externals/loky/process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


In [ ]:
print(mean_test_r2)
print(mean_test_neg_mse)

# cv_results = xgb_random_search.cv_results_
# print(cv_results)

# mean_test_r2 = cv_results['mean_test_r2_score']
# print(mean_test_r2)
# mean_test_mse = -cv_results['mean_test_neg_mean_squared_error']
# print(mean_test_mse)

[0.99509449 0.99509335 0.99518175 0.99481417 0.99505168 0.99509227
 0.99505501 0.99510916 0.99513346 0.99513669 0.9951675  0.99504084
 0.9949353  0.99520398 0.99518487 0.99513593 0.99509822 0.99381216
 0.99517547 0.99506337 0.99511101 0.99515137 0.99514492 0.99482451
 0.99495084 0.99508358 0.99512162 0.99514112 0.9951184  0.99489773
 0.99504532 0.99504595 0.99517204 0.99506776 0.99517121 0.99521261
 0.9948856  0.99513352 0.994964   0.99514454 0.99514154 0.99506231
 0.99499872 0.9948908  0.99512752 0.9949987  0.99475509 0.99513459
 0.99510017 0.99507356 0.99509993 0.99500887 0.99460645 0.99518838
 0.99502009 0.99511398 0.99428622 0.99502083 0.99515396 0.9950961
 0.99498995 0.99497181 0.99510206 0.99511983 0.99513005 0.99317992
 0.99511873 0.99500711 0.99512305 0.99508053 0.99493254 0.99516094
 0.99513886 0.99516008 0.99512166 0.99516964 0.99493026 0.99514515
 0.99502689 0.99511979 0.99502395 0.99509888 0.99485655 0.99516449
 0.99520282 0.99504365 0.99509921 0.99508706 0.99511504 0.99514

In [ ]:
import matplotlib.pyplot as plt

# Extract the mean test scores for each cross-validation fold
mean_test_scores = xgb_random_search.cv_results_['mean_test_score']
mean_train_scores = xgb_random_search.cv_results_['mean_train_score']

# Extract the number of estimators for each iteration of the RandomizedSearchCV
# Note: This assumes 'n_estimators' was a parameter being tuned.
n_estimators = [params['n_estimators'] for params in xgb_random_search.cv_results_['params']]

# Sort the scores by the number of estimators for plotting
sorted_indices = np.argsort(n_estimators)
n_estimators_sorted = np.array(n_estimators)[sorted_indices]
mean_test_scores_sorted = np.array(mean_test_scores)[sorted_indices]
mean_train_scores_sorted = np.array(mean_train_scores)[sorted_indices]

# Plot the learning curves
plt.figure(figsize=(12, 6))
plt.plot(n_estimators_sorted, mean_train_scores_sorted, label='Train Score')
plt.plot(n_estimators_sorted, mean_test_scores_sorted, label='Test Score', linestyle='--')
plt.xlabel('Number of Estimators')
plt.ylabel('R2 Score')
plt.title('Learning Curves')
plt.legend()
plt.show()

KeyError: 'mean_train_score'